In [18]:
import torch

model = torch.nn.RNN(300, 512)

In [19]:
print(torch.__version__)

2.6.0+cu124


# Recurrent Neural Networks(RNN)

In [1]:
import fastai  # 여기서 subprocess error 같은게 떠서 anaconda, python 다 삭제하고 재설치 후 가상환경 나누고 다시 설치
fastai.__version__

'2.8.0'

In [2]:
from fastai.text.all import *

In [3]:
class RNNCell(nn.Module):

    def __init__(self, input_size, hidden_size):
        super(RNNCell, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.ih = nn.Linear(input_size, hidden_size)
        self.hh = nn.Linear(hidden_size, hidden_size)

    def forward(self, x, h = None):
        if h is None:
            h = torch.zeros(x.size(0), self.hidden_size)
        h = torch.tanh(self.ih(x) + self.hh(h))
        return h

In [4]:
class RNN(nn.Module):

    def __init__(self, input_size, hidden_size):
        super(RNN, self).__init__()
        self.cell = RNNCell(input_size, hidden_size)

    def forward(self, x, h = None):

        print(x.shape)
        for i in range(x.shape[1]):
            h = self.cell(x[:,i], h)

        return h

In [5]:
class TextClassifier(nn.Module):

    def __init__(self, vocab_size, hidden_size):
        super(TextClassifier, self).__init__()
        self.emb = nn.Embedding(vocab_size, hidden_size)
        self.rnn = RNN(hidden_size, hidden_size)
        self.fc1 = nn.Linear(hidden_size, 10)
        self.fc2 = nn.Linear(10, 1)

    def forward(self, x):

        x = self.emb(x)
        x = self.rnn(x)
        x = self.fc1(x)
        out = self.fc2(x)

        return out

In [3]:
path = untar_data(URLs.IMDB)

In [7]:
path

Path('C:/Users/user/.fastai/data/imdb')

In [ ]:
dls = TextDataLoaders.from_folder(path, valid='test', bs=256)
# AttributeError: module 'torch' has no attribute 'version' 떠서 알고보니 직접 깔지도 않았던 spacy 문제.
# 알고보니 fastai 설치하면 자동으로 spacy가 설치됨(spacy-3.8.4-cp312)
# 가상환경을 새로 만들면서 어쩌다보니 spacy는 해결됐고 이제 tok 파일이 없어서 에러남. 노션 참조

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\user\\.fastai\\data\\imdb_tok\\counter.pkl'

In [6]:
dls.show_batch(max_n=5)

NameError: name 'dls' is not defined

In [ ]:
learn = Learner(dls, TextClassifier(len(dls.vocab[0]), 100),
                loss_func=CrossEntropyLossFlat(),
                metrics=accuracy)

In [ ]:
learn.fit(10)

In [ ]:
class TextClassifier(nn.Module):

    def __init__(self, vocab_size, hidden_size):
        super(TextClassifier, self).__init__()
        self.emb = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, 10)
        self.fc2 = nn.Linear(10, 2)

    def forward(self, x):

        x = self.emb(x)
        _, x = self.rnn(x)
        x = self.fc1(x)
        out = self.fc2(x)

        return out

In [ ]:
learn = Learner(dls, TextClassifier(len(dls.vocab[0]), 100),
                loss_func=CrossEntropyLossFlat(),
                metrics=accuracy)
learn.fit(10)

# Bidirectional RNN

In [ ]:
class TextClassifier(nn.Module):

    def __init__(self, vocab_size, hidden_size):
        super(TextClassifier, self).__init__()
        self.emb = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size,
                          bidirectional=True, batch_first=True)
        self.fc1 = nn.Linear(hidden_size * 2, 10)
        self.fc2 = nn.Linear(10, 2)

    def forward(self, x):

        x = self.emb(x)
        _, x = self.rnn(x)
        x = torch.cat((x[0], x[1]), dim=-1)
        x = self.fc1(x)
        out = self.fc2(x)

        return out

In [ ]:
learn = Learner(dls, TextClassifier(len(dls.vocab[0]), 100),
                loss_func=CrossEntropyLossFlat(),
                metrics=accuracy)
learn.fit(10)

# Long Short-Term Memory(LSTM)

In [ ]:
class TextClassifier(nn.Module):

    def __init__(self, vocab_size, hidden_size):
        super(TextClassifier, self).__init__()
        self.emb = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, 10)
        self.fc2 = nn.Linear(10, 2)

    def forward(self, x):

        x = self.emb(x)
        x, _ = self.rnn(x)[1]
        x = self.fc1(x)
        out = self.fc2(x)

        return out

In [ ]:
learn = Learner(dls, TextClassifier(len(dls.vocab[0]), 100),
                loss_func=CrossEntropyLossFlat(),
                metrics=accuracy)
learn.fit(10)

# Gated Recurrent Units(GRU)

In [ ]:
class TextClassifier(nn.Module):

    def __init__(self, vocab_size, hidden_size):
        super(TextClassifier, self).__init__()
        self.emb = nn.Embedding(vocab_size, hidden_size)
        self.rnn = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, 10)
        self.fc2 = nn.Linear(10, 2)

    def forward(self, x):

        x = self.emb(x)
        _, x = self.rnn(x)
        x = self.fc1(x)
        out = self.fc2(x)

        return out

In [ ]:
learn = Learner(dls, TextClassifier(len(dls.vocab[0]), 100),
                loss_func=CrossEntropyLossFlat(),
                metrics=accuracy)
learn.fit(10)